In [ ]:
import sys,os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from dashrisk import var_models as varm
from dashrisk import option_models as opmod
from dashrisk import build_history as bh
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime

### Show possible portfolio csv files

In [ ]:
print(list(filter(lambda d: '.csv' in str(d).lower(),os.listdir('.'))))

In [ ]:
df = pd.read_csv('./fxe_portfolio.csv')
df

In [ ]:
USE_POSTGRES=True
if USE_POSTGRES:
    history_fetcher = varm.PostgresFetcher(history_builder=bh.HistoryBuilder())
else:
    history_fetcher = varm.YahooFetcher()

vm = varm.VarModel(df,history_fetcher)

In [ ]:
var_dict = vm.compute_var()
port_var = var_dict['port_var']
sp_dollar_equiv = var_dict['sp_dollar_equiv'] 
print('port_var',port_var,'sp_dollar_equiv',sp_dollar_equiv)

## How did we get this VaR?

### Here's the portfolio

In [ ]:
vm.df_portfolio.sort_values(['underlying','symbol'])

In [ ]:
var_dict.keys()

### Here's the risk factors for each position

In [ ]:
var_dict['df_positions_all']

### Here are the At-The-Money (atm) prices that we use to calculate per position VaR's

In [ ]:
var_dict['df_atm_price']

### Here's the correlation matrix used to create a portfolio VaR

In [ ]:
df_hm = vm.get_history_matrix()
df_hm.drop(columns=['date'])
df_corr = df_hm.corr()
df_corr = df_corr.sort_index()
df_corr = df_corr[sorted(df_corr.columns.values)]
df_corr

### Below, we do the math to create a portfolio variance and then take its square root to get the port_var

In [ ]:
df_underlying_positions = var_dict['df_underlying_positions'].sort_values('underlying')
port_variance = df_underlying_positions.position_var.astype(float).as_matrix().T @ df_corr.astype(float).as_matrix() @ df_underlying_positions.position_var.astype(float).as_matrix()
port_var = port_variance**.5 
port_var


### Show how to fetch history for one symbol

In [ ]:
df_hist = history_fetcher.fetch_history('FXB',datetime.datetime.now()-datetime.timedelta(120),datetime.datetime.now())